# Queries Neo4j

Análises no dataset de filmes do IMDb realizadas no Neo4j

# Carregando o csv

São carregados 85 mil filmes do csv

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/HVN-MC536/projeto/main/stage04/data/processed/movies.csv' AS line
CREATE (:All {title: line.original_title, votes: line.votes, country : line.country, year : line.year, genre : line.genre})
```

Dado a limitação do Neo4j com um número tão grande de nós, são realizados queries para selecionar alguns. \

São selecionados os filmes dos países Brasil, EUA e França

```cypher
MATCH (n)
WHERE n.country = "USA" OR n.country = "Brazil" OR n.country = "France"
CREATE (:Movie {title: n.title, votes: n.votes, country : n.country, year : n.year, genre : n.genre})

MATCH (n:All) 
DELETE n
```
Assim o grafo fica com 30 mil nós, para diminuir mais, é selecionado filmes feitos entre 1985 e 2015, pois são os anos que temos os dados de suícido.

```cypher
MATCH (n)
WHERE  TOINTEGER(n.year) < 1985 OR TOINTEGER(n.year) > 2015
DELETE n
```

Restando 16 mil nós, agora será selecionado apenas os gêneros Comédai e Drama, pois podem ser mais relevantes para a análise

```cypher
MATCH (n)
WHERE n.genre <> "Comedy" AND n.genre <> "Drama"
DELETE n
```
Com ainda 3 mil nós, é retirado filmes que tiveram entre 200 e 200000 e entre 300000 e 400000 votações no IMDb, selecionando apenas a faixa abaixo de 200, entre 200000 e 300000 e acima de 400000.

```cypher
MATCH (n)
WHERE TOINTEGER(n.votes) > 200 AND TOINTEGER(n.votes) < 200000 
DELETE n

MATCH (n)
WHERE TOINTEGER(n.votes) > 300000 AND TOINTEGER(n.votes) < 400000 
DELETE n
```

Assim ficamos com 233 nós para analisar.

# Agrupando filmes semelhantes

Iremos agrupar os filmes em atributos semelhantes para facilitar a verificação de algum padrão. \

São criado nós para cada atributo.

```cypher
CREATE (:Genre {name : "Drama"})
CREATE (:Genre {name : "Comedy"})

CREATE (:Country {name : "USA"})
CREATE (:Country {name : "Brazil"})
CREATE (:Country {name : "France"})

CREATE (:Year {range : "1985 a 1995"})
CREATE (:Year {range : "1995 a 2005"})
CREATE (:Year {range : "2005 a 2015"})

CREATE (:Popularity {name : "Alta"})
CREATE (:Popularity {name : "Media"})
CREATE (:Popularity {name : "Baixa"})
```
Querie para agrupar por gênero
```cypher
MATCH (m:Movie)
MATCH (g:Genre)
WHERE m.genre = g.name
CREATE (m)-[:InGenre]->(g)
```

Querie para agrupar por País
```cypher
MATCH (m:Movie)
MATCH (c:Country)
WHERE m.country = c.name
CREATE (m)-[:Nationality]->(c)
```

Queries para agrupar por ano produzido
```cypher
MATCH (m:Movie)
MATCH (y:Year)
WHERE TOINTEGER(m.year) >=1985 AND TOINTEGER(m.year) < 1995 AND y.range = "1985 a 1995"
CREATE (m)-[:YearRangeIs]->(y)

MATCH (m:Movie)
MATCH (y:Year)
WHERE TOINTEGER(m.year) >=1995 AND TOINTEGER(m.year) < 2005 AND y.range = "1995 a 2005"
CREATE (m)-[:YearRangeIs]->(y)

MATCH (m:Movie)
MATCH (y:Year)
WHERE TOINTEGER(m.year) >=2005 AND TOINTEGER(m.year) <= 2015 AND y.range = "2005 a 2015"
CREATE (m)-[:YearRangeIs]->(y)
```

Queries para agrupar por popularidade, baseado em número de votos recebidos no IMDb
```cypher
MATCH (m:Movie)
MATCH (p:Popularity)
WHERE TOINTEGER(m.votes) < 200000 AND p.name = "Baixa"
CREATE (m)-[:PopularityIs]->(p)

MATCH (m:Movie)
MATCH (p:Popularity)
WHERE TOINTEGER(m.votes) >=200000 AND TOINTEGER(m.votes) < 400000 AND p.name = "Media"
CREATE (m)-[:PopularityIs]->(p)

MATCH (m:Movie)
MATCH (p:Popularity)
WHERE TOINTEGER(m.votes) >=400000 AND p.name = "Alta"
CREATE (m)-[:PopularityIs]->(p)
```

Assim temos todos os grupos

# Visualizando em detalhes

Para facilitar a visualização será recortado alguns grupos para análise\

Estados Unidos
```cypher
MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "2005 a 2015" AND p.name = "Alta" AND c.name = "USA"
RETURN n,y,p,c,g

MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "1995 a 2005" AND p.name = "Alta" AND c.name = "USA"
RETURN n,y,p,c,g

MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "1985 a 1995" AND p.name = "Alta" AND c.name = "USA"
RETURN n,y,p,c,g
```
É possível verificar que entre 1985 e 1995 (maior taxa de suicído), foi lançado "Sociedade dos Poetas Mortos" e possui popularidade Alta\

França
```cypher
MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "2005 a 2015" AND p.name = "Baixa" AND c.name = "France"
return n,y,p,c,g

MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "1995 a 2005" AND p.name = "Baixa" AND c.name = "France"
return n,y,p,c,g

MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "1985 a 1995" AND p.name = "Baixa" AND c.name = "France"
return n,y,p,c,g
```
Entre 1985 e 1995 (também maior taxa de suicídios), foi lançado um filme chamado "Le jeune Werther"(Jovem Werther)

Brasil
```cypher
MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "2005 a 2015" AND p.name = "Baixa" AND c.name = "Brazil"
return n,y,p,c,g

MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "1995 a 2005" AND p.name = "Baixa" AND c.name = "Brazil"
return n,y,p,c,g

MATCH (n)-[:YearRangeIs]->(y:Year)
MATCH (n)-[:PopularityIs]->(p:Popularity)
MATCH (n)-[:Nationality]->(c:Country)
MATCH (n)-[:InGenre]->(g:Genre)
WHERE y.range = "1985 a 1995" AND p.name = "Baixa" AND c.name = "Brazil"
return n,y,p,c,g
```
Brasil não teve filmes com enredo tão impactante e uma proporção mais parecida entre drama e comédia.